In [12]:
def get_apartments(craigs_url, search_distance = None, postal = None
                   , min_price = None, max_price = None, min_bedrooms = None, min_bathrooms = None):
    
    import googlemaps
    import pandas as pd
    from requests import get
    from datetime import datetime
    from bs4 import BeautifulSoup
    from time import sleep
    import re
    from random import randint #avoid throttling by not sending too many requests one after the other
    from warnings import warn
    from time import time
    from IPython.core.display import clear_output
    import numpy as np
    
    def parse_time(time):
        if not pd.isna(time.strip()):
            mins = 0
            fields=re.findall(r'[A-Za-z]+|\d+', time.strip())
            #print(fields) #inserted this line to debug why output was 0
            for idx in range(0, len(fields)-1):
                if fields[idx+1] in ('min', 'mins', 'minutes'):
                    mins += int(fields[idx])
                elif fields[idx+1] in ('h', 'hour', 'hours'):
                    mins += int(fields[idx]) * 60

            return mins
    #find the total number of posts to find the limit of the pagination
    # results_num = html_soup.find('div', class_= 'search-legend')
    # results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

    #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
    #pages = np.arange(0, results_total+1, 120)
    MY_KEY = "<YOUR-KEY>"
    appartment_counter =0
    iterations = 0

    post_timing = []
    post_hoods = []
    post_title_texts = []
    bedroom_counts = []
    sqfts = []
    post_links = []
    post_prices = []
    walking_time = []
    biking_time = []
    public_transit_time = []

    #for page in pages:

    #get request
    response = get(str(craigs_url))

    sleep(randint(1,5))

    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')

    #define the posts
    posts = page_html.find_all('li', class_= 'result-row')

    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            print("appartment " + str(appartment_counter + 1) + " posting data done!")

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            print("appartment " + str(appartment_counter + 1) + " neighborhood done!")



            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)
            print("appartment " + str(appartment_counter + 1) + " title text done!")


            #post link
            post_link = post_title['href']
            post_links.append(post_link)
            print("appartment " + str(appartment_counter + 1) + " link done!")



            ###get deeper into the link
            #get address
            post_response = get(post_link)

            page_html_soup = BeautifulSoup(post_response.text, 'html.parser')

            ## lat and long
            ## lat and long
            latlonglist = page_html_soup.find_all(attrs={"data-latitude": True, "data-longitude": True})
            for latlong in latlonglist:
                latitude = latlong['data-latitude']
                longtitude = latlong['data-longitude']
            latlon = latitude + ', ' + longtitude

            ## use google map find walking, biking and transit duration

            UIC_LAT_LONG = "41.869167477453374, -87.647870435717"


            gmaps = googlemaps.Client(key = MY_KEY)


            now = datetime.now()
            walking_result = gmaps.directions(UIC_LAT_LONG,
                                                 latlon,
                                                 mode="walking"
                                                )
            biking_result = gmaps.directions(UIC_LAT_LONG,
                                                 latlon,
                                                 mode="bicycling"
                                                )

            bus_result = gmaps.directions(UIC_LAT_LONG,
                                                 latlon,
                                                 mode="transit"
                                                )
            if not walking_result:
                walking_time.append(np.nan)
            else: 
                walking_time.append(walking_result[0]['legs'][0]['duration']['text'])

            if not biking_result:
                biking_time.append(np.nan)
            else:
                biking_time.append(biking_result[0]['legs'][0]['duration']['text'])

            if not bus_result: 
                public_transit_time.append(np.nan)
            else:
                public_transit_time.append(bus_result[0]['legs'][0]['duration']['text'])

            print("appartment " + str(appartment_counter + 1) + " walking, biking, bus done!")






            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = post.a.text.strip().replace("$", "").replace(",", "")
            post_prices.append(post_price)
            print("appartment " + str(appartment_counter + 1) + " price done!")


            if post.find('span', class_ = 'housing') is not None:

                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:

                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)

                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)

                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:

                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)

                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)

                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:

                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)

                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    

                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)

                    sqft = np.nan
                    sqfts.append(sqft)

            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)

                sqft = np.nan
                sqfts.append(sqft)
            #    bedroom_counts.append(bedroom_count)

            #    sqft = np.nan
            #    sqfts.append(sqft)
            print("appartment " + str(appartment_counter + 1) + " posting data done!")


            appartment_counter += 1
            print("appartment " + str(appartment_counter) + " scraped successfully!")

            print("\n")
    # iterations += 1
    # print("Page " + str(iterations) + " scraped successfully!")

    print("\n")

    print("Scrape complete!")
    
    chicago_crag_apts = pd.DataFrame({'posted_time':post_timing,
                                 'neighborhood':post_hoods,
                                 'post_title':post_title_texts,
                                 'bedrooms':bedroom_counts,
                                 'sqfts':sqfts,
                                 'price':post_prices,
                                 'walking_time':walking_time,
                                 'biking_time':biking_time,
                                 'public_transit_time':public_transit_time,
                                 'links':post_links})
    chicago_crag_apts['biking_time'] = chicago_crag_apts['biking_time'].map(parse_time)
    chicago_crag_apts['public_transit_time'] = chicago_crag_apts['public_transit_time'].map(parse_time)
    chicago_crag_apts['walking_time'] = chicago_crag_apts['walking_time'].map(parse_time)
    
    return chicago_crag_apts

In [13]:
test = get_apartments('https://chicago.craigslist.org/search/hhh?hasPic=1&search_distance=2&postal=60607&min_price=1000&max_price=2000&min_bedrooms=2&min_bathrooms=2&availabilityMode=0&sale_date=all+dates')

appartment 1 posting data done!
appartment 1 neighborhood done!
appartment 1 title text done!
appartment 1 link done!
appartment 1 walking, biking, bus done!
appartment 1 price done!
appartment 1 posting data done!
appartment 1 scraped successfully!


appartment 2 posting data done!
appartment 2 neighborhood done!
appartment 2 title text done!
appartment 2 link done!
appartment 2 walking, biking, bus done!
appartment 2 price done!
appartment 2 posting data done!
appartment 2 scraped successfully!


appartment 3 posting data done!
appartment 3 neighborhood done!
appartment 3 title text done!
appartment 3 link done!
appartment 3 walking, biking, bus done!
appartment 3 price done!
appartment 3 posting data done!
appartment 3 scraped successfully!


appartment 4 posting data done!
appartment 4 neighborhood done!
appartment 4 title text done!
appartment 4 link done!
appartment 4 walking, biking, bus done!
appartment 4 price done!
appartment 4 posting data done!
appartment 4 scraped successf

In [14]:
test

,posted_time,neighborhood,post_title,bedrooms,sqfts,price,walking_time,biking_time,public_transit_time,links
0,2022-03-07 17:02,(Chicago),This condo-quality building is a must see.,2,1081.0,2000,14,5,14,https://chicago.craigslist.org/chc/sub/d/chica...
1,2022-03-02 17:34,(University Commons),Spacious 1 Bedroom + Den with closet(can be us...,2,900.0,1950,14,5,14,https://chicago.craigslist.org/chc/apa/d/chica...
2,2022-02-28 12:40,"(Pilsen, UIC, Medical District)",Amazing 2 BR/2Bath duplex across from historic...,2,1200.0,1595,30,11,28,https://chicago.craigslist.org/chc/apa/d/chica...
3,2022-03-01 23:44,(Pilsen),"PILSEN Top Floor 2BR 2BA-In-unit Laundry, Quar...",2,900.0,1850,28,9,20,https://chicago.craigslist.org/chc/apa/d/chica...
4,2022-03-02 02:06,(Pilsen),"BRAND NEW 2 Bed 2 Baths, PRIME PILSEN In-unit ...",2,NaN,1850,29,10,19,https://chicago.craigslist.org/chc/apa/d/chica...
...,...,...,...,...,...,...,...,...,...,...
142,2022-03-18 07:37,(Chicago South),SECTION 8 Only!!! Spacious 4 bedrm 2 bathroom ...,4,NaN,1850,206,59,66,https://chicago.craigslist.org/sox/apa/d/chica...
143,2022-03-06 13:38,(Little India),Little India Privat Bedroom with En-suite,2,NaN,1000,209,65,74,https://chicago.craigslist.org/chc/roo/d/chica...
144,2022-03-02 10:49,(East Rodger park),7 beds and 3 beds for rent,7,NaN,1900,197,62,51,https://chicago.craigslist.org/chc/apa/d/chica...
145,2022-03-20 10:14,(East Rogers Park),Heat & Gas Included! Large Floorplan! Walk To ...,2,NaN,1450,204,64,50,https://chicago.craigslist.org/chc/apa/d/chica...
